## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

## Download

Retrieve the page

In [4]:
url = "https://itwebservices.placer.ca.gov/coviddashboard/#cases-by-zip-coderegion"

In [5]:
page = requests.get(url, verify=False)

/home/runner/.local/share/virtualenvs/california-coronavirus-scrapers-dxuBXRsm/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'itwebservices.placer.ca.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## Parse

In [6]:
soup = BeautifulSoup(page.content, "html.parser")

Find script tag with the Infogram data

In [7]:
for elem in soup(text=re.compile(r"Foresthill")):
    script_content = elem

In [8]:
data = json.loads(script_content)

NameError: name 'script_content' is not defined

In [9]:
zip_cities_list = data["x"]["calls"][1]["args"][-3]

In [10]:
dict_list = []

In [11]:
TAG_RE = re.compile(r"<[^>]+>")

In [12]:
for z in zip_cities_list:
    split_zips = z.split("<br/>")
    area = TAG_RE.sub("", str(split_zips[0]))
    zip_code, city = area.split(" - ")
    clean_cases = re.sub("Number of cases: ".lower(), "", str(split_zips[1]).lower())
    d = dict(area=area, city=city, zip_code=zip_code, confirmed_cases=clean_cases)
    dict_list.append(d)

In [13]:
df = pd.DataFrame(dict_list)

Get timestamp

In [14]:
time_div = soup.find("div", id="dashboard-data-last-updated")

In [15]:
timestamp = time_div.find("span", class_="value-output").get_text()

In [16]:
latest_date = pd.to_datetime(timestamp).date()

In [17]:
df["county_date"] = latest_date

In [18]:
df.insert(0, "county", "Placer")

Clean up

In [19]:
df["area"] = df["area"].str.replace(" - ", ": ")

In [20]:
df = df[["county", "area", "confirmed_cases", "county_date", "zip_code"]].rename(
    columns={"zip_code": "zip"}
)

## Vet

In [21]:
try:
    assert not len(df) < 32
except AssertionError:
    raise AssertionError("Placer County's scraper is missing rows")

In [22]:
try:
    assert not len(df) > 32
except AssertionError:
    raise AssertionError("Placer County's scraper has more rows than before")

## Export

Set date

In [23]:
tz = pytz.timezone("America/Los_Angeles")

In [24]:
today = datetime.now(tz).date()

In [25]:
slug = "placer"

In [26]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [27]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [28]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [29]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [30]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)